In [1]:
%run ./../../setup.py
ds = Dataset('object_detection/photoscan_projects/rmz_nexity/photoscan/cloud_chunks')

In [14]:
import laspy as laspy
import h5py
from geojson import Point, Feature, FeatureCollection, dump

In [6]:
def create_normalized_blocks(path, num_points):
    files = os.listdir(path)
    source_file = []
    normalized_blocks = N.zeros((0,num_points,3))
    block_min_array = N.zeros((0,3))
    block_max_array = N.zeros((0,3))
    for file in files:
        inFile = laspy.file.File(os.path.join(path,file), mode = 'r')
        unscaled_points = N.vstack([inFile.X, inFile.Y, inFile.Z]).transpose()
        if(len(list(unscaled_points))>=num_points):
            
            head = inFile.header
            scale = head.scale
            offset = head.offset
            points = unscaled_points * scale + offset
            block_min = head.min
            block_max = head.max
            
            #sample
            random_indexes = N.random.randint(0, high=len(points)-1, size=num_points)
            block = points[random_indexes]
            
            #normalize
            #block_min = N.min(block ,axis = 0)
            #block_max = N.max(block ,axis = 0)
            
            points_sub = block - block_min
            diff = N.array(block_max) - N.array(block_min)
            n_block = points_sub/diff
            normalized_blocks = N.append(normalized_blocks,[n_block],axis=0)
            block_min_array = N.append(block_min_array,[block_min],axis=0)
            block_max_array = N.append(block_max_array,[block_max],axis=0)
            source_file.append(file)
                                    
    return normalized_blocks,source_file,block_min_array,block_max_array   

In [4]:
# file has string data!!!!

def write_block_to_h5(filename, data, file, block_min_array, block_max_array):
    hf = h5py.File(filename, 'w')
    hf.create_dataset('data', data=data)
    asciiList = [n.encode("ascii", "ignore") for n in file]
    string_type = h5py.special_dtype(vlen=bytes)
    hf.create_dataset('source_file', shape = (len(asciiList),1), data = asciiList, dtype=string_type)
    hf.create_dataset('min',data = block_min_array)
    hf.create_dataset('max',data = block_max_array)
    hf.close()

In [9]:
def loadBlockData(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    files = f['source_file'][:]
    return (data, files)


In [12]:
def find_block(filename, query_x, query_y):
    f = h5py.File(filename)
    files = f['source_file'][:]
    min_data = f['min'][:]
    max_data = f['max'][:]
    feature_collection = []
    for i in range(min_data.shape[0]):
        if(min_data[i][0]<query_x<max_data[i][0] and min_data[i][1]<query_y<max_data[i][1]):
            print(files[i][0].decode("utf-8"))
            point1 = Point((min_data[i][0],min_data[i][1],min_data[i][2]))
            point2 = Point((max_data[i][0],max_data[i][1],max_data[i][2]))
                
            f1 = Feature(geometry=point1)
            feature_collection.append(f1)
            f2 = Feature(geometry=point2)
            feature_collection.append(f2)

            fc = FeatureCollection(feature_collection)
            with open('min_max.geojson', 'w') as f:
                dump(fc, f)    
            return files[i][0].decode("utf-8")

In [7]:
num_points = 1024
normalized_blocks, source_file, block_min_array, block_max_array = create_normalized_blocks(ds.path(),num_points)
print(normalized_blocks.shape)

(3252, 1024, 3)


In [8]:
write_block_to_h5('block_rmz.h5',normalized_blocks,source_file,block_min_array, block_max_array)

In [10]:
block_df = P.read_pickle('feature_0.pkl')
#feature = block_df['feature']
feature_array = N.load('feature_0.npy')
print(feature_array.shape)

(3252, 1024)


In [15]:
query_x = 5066.69
query_y = 7849.21

from sklearn.neighbors import KDTree
#tree = KDTree(block_df['feature'].as_matrix())  
tree = KDTree(feature_array)
block_file = find_block('block_rmz.h5',query_x, query_y)
index = block_df['file'] == block_file
dist, ind = tree.query(feature_array[index].reshape(1,-1), k=10)                

truck_3.las


/mnt/research-disk/virtualenvs/tf-cpu1/lib/python3.6/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [16]:
print(ind)

[[1928  748  719 2973 1930 2821 1411 1373 2395 2403]]


In [17]:
nn_files = list(block_df['file'][ind[0]])
print(nn_files)

['truck_3.las', 'truck_4.las', 'split_5-14-68.las', 'split_5-45-33.las', 'truck_1.las', 'split_5-17-80.las', 'truck_2.las', 'split_5-58-41.las', 'split_5-41-28.las', 'split_5-27-45.las']


In [21]:

feature_collection = []
nn_files = list(block_df['file'][ind[0]])
idx=0
for file in nn_files:
    inFile = laspy.file.File(os.path.join(ds.path(),file), mode = 'r')
    unscaled_points = N.vstack([inFile.X, inFile.Y, inFile.Z]).transpose()
    head = inFile.header
    scale = head.scale
    offset = head.offset
    points = unscaled_points * scale + offset
    center = N.mean(points,axis=0)
    idx=idx+1
    point = Point(list(center))    
    feature = Feature(geometry=point,id=idx)
    feature_collection.append(feature)
    
fc = FeatureCollection(feature_collection)
with open('point_trucks.geojson', 'w') as f:
    dump(fc, f)    

In [45]:
print(center)

[4983.84012703 7768.05703727  591.26713637]


In [ ]:
from utils import pc_util
import imageio

for i in range(normalized_blocks.shape[0]):
    img = pc_util.point_cloud_three_views(normalized_blocks[i,:,:])
    filename = os.path.join('./images',source_file[i].split('.')[0]+'.jpg')
    imageio.imwrite(filename,img)